## Basic code using spaudiopy package
- Docs: https://spaudiopy.readthedocs.io/en/latest/index.html
- GitHub: https://github.com/chris-hld/spaudiopy

In [1]:
# Check spaudiopy.sig functions to open MONO signal and play with HRIRs
import spaudiopy as spa
from spaudiopy.sig import MonoSignal as ms
from spaudiopy.sig import MultiSignal as stereo 
import spaudiopy.process as sproc
from scipy.io import wavfile
from scipy.io import wavfile
import scipy.signal
from IPython.display import Audio
fs=44100
import numpy as np

## Tools
We use three parameters to locate an HRTF:
1. *Azimuth*: angle between position and sound location on the $xy$-plane
2. *Elevation*: angle between position and sound location on the $xz$-plane
3. *Time* or *Frequency*: time period or frequency of emitted sound w.r.t. actual position

Math tools:
- *Haversine distance*: Consider two points $x_1$ and $x_2$ on a sphere with respective latitudes and longitudes $(\varphi_1,\varphi_2)$ and $(\theta_1,\theta_2)$. The Haversine distance $D(x_1,x_2)$ is the angular distance between them on the surface of the sphere given by $$D(x_1,x_2)=2\arcsin\sqrt{\sin^2\left(\frac{\varphi_2-\varphi_1}{2}\right)+\cos x_1\cos y_1\sin^2\left(\frac{\theta_2-\theta_1}{2}\right)}.$$ We use this distance when wanting to find the closest HRIR point from a grid.

In [2]:
piano_test = ms.from_file("piano_testfile.wav")
piano_test # keep first 3 seconds of file
piano_test.play()

In [4]:
# Helper functions
def stereo_to_mono_v2(signal2D, normalize=True):
    return list(map(lambda x: int(x[0]/2+x[1]/2), signal2D))
    
    
# get some sound file
samplerate, sample_song_stereo = wavfile.read('sample1.wav') # samplerate = fs
sample_song_mono = stereo_to_mono_v2(sample_song_stereo[:int(len(sample_song_stereo)/16)]) # too long, cut audio

display(Audio(sample_song_mono, rate=fs))

In [5]:
#delay = sproc.lagrange_delay(10000, delay=12)
delay = np.zeros(336520, dtype=int)
delay[int(fs*0.03)] = 1

In [6]:
print(len(sample_song_mono))

336520


In [7]:
delayed_sample = scipy.signal.convolve(sample_song_mono, delay)

In [8]:
display(Audio(delayed_sample, rate=fs))

In [9]:
def fusion(s1, s2):
    s = []
    for i in range(len(s1)):
        s.append(int(s1[i]/2 + s2[i]/2))
    return s

In [10]:
display(Audio(fusion(sample_song_mono,delayed_sample), rate=fs))

test delay on stereo file

In [36]:
stereo_sample = stereo.from_file("sample1.wav")
stereo_sample.trim(0,8)
stereo_sample.play()

In [19]:
s1, s2 = stereo_sample.get_signals()
display(Audio(s2, rate=fs))

In [47]:
delay = np.zeros(len(stereo_sample), dtype=int)
delay[int(fs*0.03)] = 1
delayed_s2 = scipy.signal.convolve(s2, delay)[:len(stereo_sample)]

In [48]:
delayed_stereo = stereo([s1, delayed_s2], fs=fs)
delayed_stereo.play()